In [9]:
!git clone https://github.com/SobhanRjz/DL_Detectron2.git
%cd DL_Detectron2

fatal: destination path 'DL_Detectron2' already exists and is not an empty directory.
/content/DL_Detectron2


In [3]:
# prompt: get from drive folder

from google.colab import drive
drive.mount('/content/drive')
# Then you can use standard file operations to access the files.
# For example, to list the files in a folder:
!ls "/content/drive/My Drive/Colab Notebooks"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 images.rar   NeuralNetwork.ipynb  'Train_Detectron2_(7)_(9)_(1)_(6).ipynb'   Untitled0.ipynb


In [4]:
# prompt: makedir if not exist /content/DL_Detectron2/DataSets/images
import os
# Create the directory if it does not exist
dir_path = "/content/DL_Detectron2/DataSets/images"
if not os.path.exists(dir_path):

    os.makedirs(dir_path)
    print(f"Directory '{dir_path}' created successfully.")
else:
    print(f"Directory '{dir_path}' already exists.")

Directory '/content/DL_Detectron2/DataSets/images' already exists.


In [ ]:
# prompt: unzip rar file

!unrar x "/content/drive/My Drive/Colab Notebooks/images.rar" "/content/DL_Detectron2/DataSets/images"


In [6]:

# Look at training curves in tensorboard:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

from google.colab.patches import cv2_imshow



  Using cached PyYAML-5.1.tar.gz (274 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15837, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 15837 (delta 30), reused 16 (delta 15), pack-reused 15780 (from 2)
Receiving objects: 100% (15837/15837), 6.40 MiB | 21.49 MiB/s, done.
Resolving deltas: 100% (11537/11537), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
nvcc: NVIDIA (R) Cuda compile

In [31]:
import logging
import sys

# Configure the logger
logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format="%(asctime)s [%(levelname)s] %(message)s",
    force=True, # Resets any previous configuration
    handlers=[
        logging.StreamHandler(sys.stdout)  # Stream logs to Colab's cell output
    ]
)

# Create logger instance
logger = logging.getLogger()
logger.info("Logger is configured!")


2025-01-14 18:44:14,878 [INFO] Logger is configured!


Processing COCO JSON files...


In [32]:
import sys

logger.info("This is an INFO log.")
sys.stdout.flush()  # Ensure output is flushed to Colab's cell

2025-01-14 18:44:28,801 [INFO] This is an INFO log.


In [26]:
!pip install -U ipykernel

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.1 MB/s eta 0:00:00
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ipykernel-5.5.6:
      Successfully uninstalled ipykernel-5.5.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.


In [33]:

import logging
import sys

# After configuring the logger
logger = logging.getLogger(__name__)
logger.info("Processing COCO JSON files...")
sys.stdout.flush()  # Ensure logs are flushed to the notebook output

2025-01-14 18:45:23,201 [INFO] Processing COCO JSON files...


In [7]:
import detectron2
print(f"Detectron2 version: {detectron2.__version__}")


Detectron2 version: 0.6


In [10]:
import os
import sys
# Get the parent directory path
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)


import logging
from pathlib import Path
import torch
from Config.basic_config import BASE_PATH, DATA_PATH, OUTPUT_PATH, DEVICE
from Config.basic_config import detectron2_logger as logger
from Config.detectron2_config import DetectronConfig
from Config.dataset_config import DatasetConfig
from PreProcess.preprocess_COCO import COCOJsonProcessor
from Train import mainTrain
from detectron2.engine import launch


In [8]:
DATA_PATH

'/content/DL_Detectron2/DataSets/images'

In [11]:
# Use pathlib for more robust path handling
json_paths = [
    Path(DATA_PATH) / split / "_annotations.coco.json"
    for split in ["train", "test", "valid"]
]

logger.info("Processing COCO JSON files...")
# coco_processor = COCOJsonProcessor([str(p) for p in json_paths])
# coco_processor.process_files()

logger.info("Registering datasets...")
DatasetConfig().register_datasets()


['coco_2014_train', 'coco_2014_val', 'coco_2014_minival', 'coco_2014_valminusminival', 'coco_2017_train', 'coco_2017_val', 'coco_2017_test', 'coco_2017_test-dev', 'coco_2017_val_100', 'keypoints_coco_2014_train', 'keypoints_coco_2014_val', 'keypoints_coco_2014_minival', 'keypoints_coco_2014_valminusminival', 'keypoints_coco_2017_train', 'keypoints_coco_2017_val', 'keypoints_coco_2017_val_100', 'coco_2017_train_panoptic_separated', 'coco_2017_train_panoptic_stuffonly', 'coco_2017_train_panoptic', 'coco_2017_val_panoptic_separated', 'coco_2017_val_panoptic_stuffonly', 'coco_2017_val_panoptic', 'coco_2017_val_100_panoptic_separated', 'coco_2017_val_100_panoptic_stuffonly', 'coco_2017_val_100_panoptic', 'lvis_v1_train', 'lvis_v1_val', 'lvis_v1_test_dev', 'lvis_v1_test_challenge', 'lvis_v0.5_train', 'lvis_v0.5_val', 'lvis_v0.5_val_rand_100', 'lvis_v0.5_test', 'lvis_v0.5_train_cocofied', 'lvis_v0.5_val_cocofied', 'cityscapes_fine_instance_seg_train', 'cityscapes_fine_sem_seg_train', 'citysca

In [12]:
from collections import Counter
from detectron2.data import DatasetCatalog, MetadataCatalog
import matplotlib.pyplot as plt
import matplotlib

# Load dataset

dataset_dicts = DatasetCatalog.get("my_dataset_train")
metadata = MetadataCatalog.get("my_dataset_train")



In [13]:
def setup_training():
    """Configure training settings and get GPU count"""
    logger.info("Loading Detectron2 configuration...")


    detectron_config = DetectronConfig()
    cfg = detectron_config.get_cfg()

    num_gpus = torch.cuda.device_count()
    logger.info(f"Number of GPUs available: {num_gpus}")

    return cfg, num_gpus
# Setup training configuration
cfg, num_gpus = setup_training()

[[156.85613942 426.09218377]
 [ 73.3173741   80.54901604]
 [410.20789239 240.90629763]
 [102.14131531 212.40040423]
 [212.87008804 117.50150553]]
Calculated Anchor Ratios: [2.17276078 6.31717508 0.4676243  3.73976146 1.15638598]
Adjusting anchor sizes from 5 to match 4 IN_FEATURES.
Total Iterations: 24.6240234375
Configuration saved to /content/DL_Detectron2/output/mask_rcnn_X_101_32x8d_FPN_3x.yaml


In [17]:
from IPython.display import display, HTML

# Make the output scrollable
display(HTML("<style>.output_scroll { height: 400px !important; overflow: auto; }</style>"))


In [ ]:
logger.info("Launching training...")
# Set a custom height for the output
from IPython.display import display, HTML

# Make the output scrollable
display(HTML("<style>.output_scroll { height: 400px !important; overflow: auto; }</style>"))


mainTrain(cfg, "default")


2025-01-14 18:45:43,564 [INFO] Launching training...


2025-01-14 18:45:43,568 [INFO] Initializing default trainer...
2025-01-14 18:45:45,260 [INFO] Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d